<a href="https://colab.research.google.com/github/tripleseven190504/change-face/blob/main/DeepFake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cài thư viện

In [ ]:
%cd /content/
!pip install --upgrade pip
!git clone https://github.com/tripleseven190504/change-face.git
!pip install fastapi kaleido python-multipart uvicorn
!pip install --force-reinstall lida gcsfs google-colab huggingface-hub imageio tensorflow tensorflow-probability torchdata torchtext yfinance
!pip install -r /content/change-face/requirements.txt
!wget https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O inswapper_128.onnx
!mkdir /content/change-face/models
!mv inswapper_128.onnx /content/change-face/models
!pip uninstall onnxruntime onnxruntime-gpu -y
!pip install torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/cu118
!pip install onnxruntime-gpu==1.15.1

# Chạy

In [ ]:
import cv2


def convert_frame_rate(input_video_path, output_video_path, target_fps=24):
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    frame_indices = [int(frame_index) for frame_index in range(
        0, frame_count, int(fps / target_fps))]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc,
                          target_fps, (frame_width, frame_height))
    for frame_index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            out.write(frame)
        else:
            break

    cap.release()
    out.release()


convert_frame_rate('input_video.mp4', 'video.mp4', target_fps=24)


In [ ]:
%cd /content/change-face
!python run.py -s /content/image.jpg -t /content/video.mp4 -o /content/new_video.mp4 --frame-processor face_swapper